训练并导出RNN模型

In [29]:


import numpy as np
import pandas as pd
from torch.utils.data import Dataset

import mylib

In [30]:
device = mylib.choose_device()

## 定义模型

In [31]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F


class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.rnn = nn.LSTM(input_size=28, hidden_size=64, batch_first=True)
        self.batchnorm = nn.BatchNorm1d(64)
        self.dropout1 = nn.Dropout2d(0.25)
        self.dropout2 = nn.Dropout2d(0.5)
        self.fc1 = nn.Linear(64, 32)
        self.fc2 = nn.Linear(32, 10)

    def forward(self, input):
        # Shape of input is (batch_size,1, 28, 28)
        # converting shape of input to (batch_size, 28, 28)
        # as required by RNN when batch_first is set True
        input = input.reshape(-1,1,1)
        output, hidden = self.rnn(input)

        # RNN output shape is (seq_len, batch, input_size)
        # Get last output of RNN
        output = output[:, -1, :]
        output = self.batchnorm(output)
        output = self.dropout1(output)
        output = self.fc1(output)
        output = F.relu(output)
        output = self.dropout2(output)
        output = self.fc2(output)
        output = F.log_softmax(output, dim=1)
        return output

In [32]:
df = pd.read_csv("../datasets/mfp/test_data.csv", dtype={'v': np.float32},index_col=0,parse_dates=True)
print(type(df[['x', 'y']].iloc[0].values[0]))

<class 'numpy.float32'>


In [33]:

def train(model, device, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 10:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                       100. * batch_idx / len(train_loader), loss.item()))


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))


In [34]:
class FlowSpeedDataset(Dataset):
    """Face Landmarks dataset."""

    def __init__(self, csv_file, transform=None):
        self.flow_speed = pd.read_csv(csv_file, dtype={'x': np.float32, 'y': np.float32})
        self.transform = transform

    def __len__(self):
        return len(self.flow_speed)

    def __getitem__(self, idx):
        item = self.flow_speed[['x', 'y']].iloc[idx].values
        if self.transform:
            item = self.transform(item)
        return tuple(item)


In [35]:
from torch.optim.lr_scheduler import StepLR

torch.manual_seed(1)
train_loader = torch.utils.data.DataLoader(
    FlowSpeedDataset("../datasets/mfp/train_data.csv"),
    batch_size=64, shuffle=True)
test_loader = torch.utils.data.DataLoader(
    FlowSpeedDataset("../datasets/mfp/test_data.csv"),
    batch_size=1000, shuffle=True)

model = Net().to(device)
optimizer = optim.Adadelta(model.parameters(), lr=0.1)

scheduler = StepLR(optimizer, step_size=1, gamma=0.7)
for epoch in range(1, 14 + 1):
    train(model=model, device=device, train_loader=train_loader, optimizer=optimizer, epoch=epoch)
    test(model=model, device=device, test_loader=test_loader)
    scheduler.step()

torch.save(model.state_dict(), "./output/model/flow_speed_rnn.pt")

RuntimeError: input.size(-1) must be equal to input_size. Expected 28, got 1